In [1]:
# import pandas as pd
# from datasets import load_dataset

# ds = load_dataset("effectiveML/ArXiv-10", "default")

# df = pd.DataFrame(ds)

# df.to_csv("arxiv-dataset.csv", index=False)

# print("Dataset saved as arxiv-dataset.csv")
# 


In [2]:
import pandas as pd
from datasets import get_dataset_config_names

df = pd.read_csv("arxiv-dataset.csv")
configs = get_dataset_config_names("effectiveML/ArXiv-10")
print(configs)



c:\Users\jagad\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['default']


In [ ]:
import ast 

df_train = pd.DataFrame(df['train'])

df_train['train'] = df_train['train'].apply(ast.literal_eval)

df_train['title'] = df_train['train'].apply(lambda x: x['title'])
df_train['abstract'] = df_train['train'].apply(lambda x: x['abstract'])
df_train['label'] = df_train['train'].apply(lambda x: x['label'])

df_train.drop(columns=['train'], inplace=True)


                                               title  \
0  The Pre-He White Dwarfs in Eclipsing Binaries....   
1  A Possible Origin of kHZ QPOs in Low-Mass X-ra...   
2  The effects of driving time scales on heating ...   
3  A new hard X-ray selected sample of extreme hi...   
4  The baryon cycle of Seven Dwarfs with superbub...   

                                            abstract     label  
0    We report the first $BV$ light curves and hi...  astro-ph  
1    A possible origin of kHz QPOs in low-mass X-...  astro-ph  
2    Context. The relative importance of AC and D...  astro-ph  
3    Extreme high-energy peaked BL Lac objects (E...  astro-ph  
4    We present results from a high-resolution, c...  astro-ph  


In [13]:
df_train.head()

,title,abstract,label
0,The Pre-He White Dwarfs in Eclipsing Binaries....,We report the first $BV$ light curves and hi...,astro-ph
1,A Possible Origin of kHZ QPOs in Low-Mass X-ra...,A possible origin of kHz QPOs in low-mass X-...,astro-ph
2,The effects of driving time scales on heating ...,Context. The relative importance of AC and D...,astro-ph
3,A new hard X-ray selected sample of extreme hi...,Extreme high-energy peaked BL Lac objects (E...,astro-ph
4,The baryon cycle of Seven Dwarfs with superbub...,"We present results from a high-resolution, c...",astro-ph


In [14]:
df_train["text"] = df_train.apply(lambda row: f"title: {row['title']}\nabstract: {row['abstract']}", axis=1)


In [15]:
df_train.head()

,title,abstract,label,text
0,The Pre-He White Dwarfs in Eclipsing Binaries....,We report the first $BV$ light curves and hi...,astro-ph,title: The Pre-He White Dwarfs in Eclipsing Bi...
1,A Possible Origin of kHZ QPOs in Low-Mass X-ra...,A possible origin of kHz QPOs in low-mass X-...,astro-ph,title: A Possible Origin of kHZ QPOs in Low-Ma...
2,The effects of driving time scales on heating ...,Context. The relative importance of AC and D...,astro-ph,title: The effects of driving time scales on h...
3,A new hard X-ray selected sample of extreme hi...,Extreme high-energy peaked BL Lac objects (E...,astro-ph,title: A new hard X-ray selected sample of ext...
4,The baryon cycle of Seven Dwarfs with superbub...,"We present results from a high-resolution, c...",astro-ph,title: The baryon cycle of Seven Dwarfs with s...


In [16]:
df_train = df_train.drop(columns=['title', 'abstract'])

In [17]:
df_train.head()

,label,text
0,astro-ph,title: The Pre-He White Dwarfs in Eclipsing Bi...
1,astro-ph,title: A Possible Origin of kHZ QPOs in Low-Ma...
2,astro-ph,title: The effects of driving time scales on h...
3,astro-ph,title: A new hard X-ray selected sample of ext...
4,astro-ph,title: The baryon cycle of Seven Dwarfs with s...


In [23]:
label2id = {label: idx for idx, label in enumerate(df_train["label"].unique())}
id2label = {idx: label for label, idx in label2id.items()}
df_train["label"] = df_train["label"].map(label2id)


In [24]:
df_train.head()

,label,text
0,0,title: The Pre-He White Dwarfs in Eclipsing Bi...
1,0,title: A Possible Origin of kHZ QPOs in Low-Ma...
2,0,title: The effects of driving time scales on h...
3,0,title: A new hard X-ray selected sample of ext...
4,0,title: The baryon cycle of Seven Dwarfs with s...


In [26]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(df_train["text"], df_train["label"], test_size=0.2, random_state=42)

train_data = Dataset.from_dict({"text": train_texts.tolist(), "label": train_labels.tolist()})
test_data = Dataset.from_dict({"text": test_texts.tolist(), "label": test_labels.tolist()})
dataset = DatasetDict({"train": train_data, "test": test_data})

In [29]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 80000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 20000
    })
})

In [30]:
model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 20000/20000 [00:09<00:00, 2022.41 examples/s]


In [33]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 80000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20000
    })
})

In [41]:
tokenized_datasets["train"][:1]


{'text': ['title: Application of single metal Au and Ag/Si3N4/Au plasmonic sensors for\n  glucose refractive index measurements\nabstract:   We have employed single gold layer metal and multilayer Ag/Si3N4/Au plasmonic\nsensors to monitor changes in the refraction index of glucose/water solutions.\nOur results show that the sensor can resolve glucose refractive index down to\n1% concentration. In addition, we have simulated the strength of the surface\nplasmon resonance evanescent electric fields by using the finite difference\nmethod of COMSOL. The electric field strength for the Ag/Si3N4/Au sensor is\nmuch stronger than that for single gold layer SPR sensor.\n'],
 'label': [7],
 'input_ids': [[102,
   10037,
   862,
   1836,
   131,
   1232,
   4353,
   3403,
   137,
   548,
   1352,
   2734,
   30138,
   30111,
   30140,
   1352,
   3403,
   24130,
   141,
   4965,
   168,
   3667,
   14479,
   2018,
   2354,
   4940,
   862,
   185,
   360,
   4208,
   1232,
   5157,
   2474,
   43